# Situación de proveedores

El siguiente programa tiene por objetivo realizar la búsqueda de una empresa en las plataformas de la Secretaría de la Función Pública (S.F.P.), Comisión Federal de Competencia Económica (COFECE) y la Procuraduría Federal del Consumidor (PROFECO). Esto se realiza para conocer la situación legal y administrativa en que se encuentran las empresas que remitieron cotizaciones.

El sistema realiza la búsqueda de forma automática por lo que inicialmente tomará el nombre de la empresa (que puede ir cambiando de acuerdo a la forma de búsqueda de cada página) y se le dan un tiempo estimado para poder tomar las capturas de pantalla para el análisis de la investigación de mercado

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pyautogui
import time

import os
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
from reportlab.pdfgen import canvas

In [2]:
listaEmpresa = []
op = 'S'
while (op == 'S') or (op == 's'):
    empresa = input("Escribe el nombre de la empresa a realizar la búsqueda: ")
    listaEmpresa.append(empresa)
    op = input("Desea agregar otra empresa? [S/N]")
tiempo_ini = 10
tiempo = input("Escribe los segundos que se quedará en la página antes de que se cierre [default: 10 seg]")
if tiempo == "":
    tiempo = tiempo_ini
else:
    tiempo = int(tiempo)

Escribe el nombre de la empresa a realizar la búsqueda: Optisoft
Desea agregar otra empresa? [S/N]S
Escribe el nombre de la empresa a realizar la búsqueda: Cognitus
Desea agregar otra empresa? [S/N]N
Escribe los segundos que se quedará en la página antes de que se cierre [default: 10 seg]2


## COFECE

Se realizará la búsqueda a través de la COFECE, muchas veces se requiere un cambio de nombre. Por lo anterior se pedirá confirmar el nombre de la empresa

In [3]:
def busquedaCofece(empresa, tiempo):
    cofece_elementos = ['uc_investigaciones_Check_sancionadas', 'uc_investigaciones_Check_pma', 'uc_investigaciones_Check_pmr']
    busqueda_e = input(f"Desea cambiar el nombre de la empresa a buscar [{empresa}] en la página de la COFECE?")
    if busqueda_e == "":
        busqueda_e = empresa

    url_cofece = "https://resoluciones.cofece.mx/"
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url_cofece)
    busqueda = driver.find_element(By.XPATH,'/html/body/form/div[3]/div[1]/div[2]/div/div[1]/div[1]/input')
    busqueda.send_keys(busqueda_e)
    time.sleep(1)
    # Hacer el screenshot
    screenshot = pyautogui.screenshot()
    screenshot.save(f"img/COFECE/{busqueda_e}-1.png")
    busqueda.send_keys("\n")
    for check in cofece_elementos:
        elementos = driver.find_element(By.ID, check)
        elementos.click()
    # Moverse al final de la pantalla
    driver.execute_script("window.scrollTo(0,390)")
    time.sleep(5)
    screenshot = pyautogui.screenshot()
    screenshot.save(f"img/COFECE/{busqueda_e}-2.png")
    time.sleep(tiempo)
    driver.quit()

## PROFECO

Búsqueda a través de la plataforma de PROFECO, Acá realiza un %*NombreEmpresa*% Por lo que puede ser necesario cambiar el nombre dado por default

In [4]:
def busquedaProfeco(empresa, tiempo):
    busqueda_e = input(f"Desea cambiar el nombre de la empresa a buscar [{empresa}] en la página de la PROFECO?")
    if busqueda_e == "":
        busqueda_e = empresa
        
    url_profeco = "https://burocomercial.profeco.gob.mx/"
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url_profeco)
    busqueda = driver.find_element(By.ID,'proveedor')
    busqueda.send_keys(busqueda_e)
    elementos = driver.find_element(By.ID, 'buscar')
    elementos.click()
    # Moverse al final de la pantalla
    time.sleep(2)
    driver.execute_script("window.scrollTo(0,1000)")
    # Hacer el Screenshot
    time.sleep(1)
    screenshot = pyautogui.screenshot()
    screenshot.save(f"img/PROFECO/{busqueda_e}.png")
    time.sleep(tiempo)
    driver.quit()

## S.F.P.

Búsqueda a través de la plataforma de la Secretaría de la Función Pública, puede ser necesario cambiar el nombre dado por default

In [5]:
def busquedaSFP(empresa, tiempo):
    busqueda_e = input(f"Desea cambiar el nombre de la empresa a buscar [{empresa}] en la página de la SFP?")
    if busqueda_e == "":
        busqueda_e = empresa
    url_sfp = "https://directoriosancionados.apps.funcionpublica.gob.mx/"
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url_sfp)
    busqueda = driver.find_element(By.ID,'textoBusqueda')
    busqueda.send_keys(busqueda_e)
    elementos = driver.find_element(By.XPATH, '/html/body/app-root/div/app-body/form/div/div[3]/div[2]/body-seleccion-prov/form/div[1]/button[1]')
    elementos.click()
    # Moverse al final de la pantalla
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    # Hacer el Screenshot
    time.sleep(1)
    screenshot = pyautogui.screenshot()
    screenshot.save(f"img/SFP/{busqueda_e}.png")
    time.sleep(tiempo)
    driver.quit()

## Generación de Archivo PDF

El siguiente código genera el archivo PDF que se empleará para añadir como evidencia de la situación de los oferentes.

In [6]:
def hacerPDF(dependecia):
    w, h = letter
    imagen_ancho = 492
    imagen_alto = 275
    if dependencia == 1:
        nombreDependencia = "Secretaría de la Función Pública"
        abrev = "SFP"
        ruta = 'img/SFP/'
    elif dependencia == 2:
        nombreDependencia = "Comisión Federal de Competencia Económica"
        abrev = "COFECE"
        ruta = 'img/COFECE/'
    else:
        nombreDependencia = "Procraduría Federal del Consumidor"
        abrev = "PROFECO"
        ruta = 'img/PROFECO/'
    
    c = canvas.Canvas(f"{abrev}.pdf", pagesize=letter)
    archivos = os.listdir(ruta)
    c.drawString(200, h-65, nombreDependencia)
    print(f"Generando archivo: {abrev}.pdf ...")
    cont = 0
    for img in archivos:
        if cont % 2 == 0:
            c.drawString(50, h - 50, "Sría. Def. Nal.")
            c.drawString(450, h - 50, "Dir. Gral. Inftca.")
            c.drawImage(ruta + img, 50, h - (imagen_alto+90), width=imagen_ancho, height=imagen_alto)
            c.drawString(250, h - (imagen_alto+105), img)
        if cont % 2 == 1:
            c.drawImage(ruta + img, 50, h - ((imagen_alto+90)*2), width=imagen_ancho, height=imagen_alto)
            c.drawString(250, h - ((imagen_alto+100)*2), img)
            c.showPage()
        cont = cont+1
    c.save()
    print("Archivo Generado.")

In [7]:
for empresa in listaEmpresa:
    busquedaCofece(empresa, tiempo)
    busquedaProfeco(empresa, tiempo)
    busquedaSFP(empresa, tiempo)

Desea cambiar el nombre de la empresa a buscar [Optisoft] en la página de la COFECE?
Desea cambiar el nombre de la empresa a buscar [Optisoft] en la página de la PROFECO?
Desea cambiar el nombre de la empresa a buscar [Optisoft] en la página de la SFP?
Desea cambiar el nombre de la empresa a buscar [Cognitus] en la página de la COFECE?
Desea cambiar el nombre de la empresa a buscar [Cognitus] en la página de la PROFECO?
Desea cambiar el nombre de la empresa a buscar [Cognitus] en la página de la SFP?


In [8]:
for dependencia in range(1,4):
    hacerPDF(dependencia)

Generando archivo: SFP.pdf ...
Archivo Generado.
Generando archivo: COFECE.pdf ...
Archivo Generado.
Generando archivo: PROFECO.pdf ...
Archivo Generado.
